Initialise!

In [27]:
using Queryverse
using DataFrames
using CSV
using PlotlyJS
using Dates

Load Traffic Data CSV!

In [28]:
traffic_data = CSV.read("Traffic/Riccarton Road 2020_09_17.csv", DataFrame)

Row,Hour,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Weekday,Weekend,All Days
,Time,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,00:00:00,109,124,146,199,151,307,328,146,318,195
2,01:00:00,36,73,89,89,75,186,239,72,213,112
3,02:00:00,37,31,52,79,61,117,188,52,153,81
4,03:00:00,48,68,36,64,47,81,151,53,116,71
5,04:00:00,60,75,72,110,88,67,94,81,81,81
6,05:00:00,155,171,183,164,156,102,95,166,99,147
7,06:00:00,502,494,464,490,487,226,172,487,199,405
8,07:00:00,858,929,917,864,890,356,241,892,299,722
9,08:00:00,1089,1122,1039,1150,1181,624,443,1116,534,950


Collect useful data

In [29]:
days = names(traffic_data)[2:8] ;
traffic_collated = stack(traffic_data, days)[:,[1,5,6]]

Row,Hour,variable,value
,Time,String,Int64
1,00:00:00,Mon,109
2,01:00:00,Mon,36
3,02:00:00,Mon,37
4,03:00:00,Mon,48
5,04:00:00,Mon,60
6,05:00:00,Mon,155
7,06:00:00,Mon,502
8,07:00:00,Mon,858
9,08:00:00,Mon,1089


Rearrange and rename columns to something reasonable

In [30]:
traffic_collated[!, [2,1,3]]
rename!(traffic_collated, :variable => :Day, :value => :Count)

Row,Hour,Day,Count
,Time,String,Int64
1,00:00:00,Mon,109
2,01:00:00,Mon,36
3,02:00:00,Mon,37
4,03:00:00,Mon,48
5,04:00:00,Mon,60
6,05:00:00,Mon,155
7,06:00:00,Mon,502
8,07:00:00,Mon,858
9,08:00:00,Mon,1089


Plot Traffic Data for Riccarton Road

In [32]:
plot(
    traffic_collated, x=:Hour, y=:Count, color=:Day,
    Layout(
        width=600,height=500,
        showlegend=true,
        xaxis=attr(dtick=2, tickformat="%H"),
        title="Daily Traffic Trend Riccarton Road"
    )
)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, legend, margin, showlegend, template, title, width, xaxis, and yaxis"

Import Power Data!

In [8]:
power_data = CSV.read("Power Usage/Zone Data (16 Sep - 12 Oct) [5 intervals].csv", DataFrame);

Adjust Data to be useable by plotting

In [9]:
rename!(power_data, Symbol("NZ TOTAL(MW)") => :Usage_MW)
power_data[!,1] = [parse(DateTime, x, dateformat"dd uuu yyy HH:MM") for x in power_data[!, 1]]
transform!(power_data, :Date => ByRow(x -> (Date(x), Time(x))) => [:Date, :Time])
power_data = power_data[!, [1,4,2]]

Row,Date,Time,Usage_MW
,Date,Time,Float64
1,2023-09-16,00:00:00,4289.53
2,2023-09-16,00:05:00,4253.13
3,2023-09-16,00:10:00,4222.4
4,2023-09-16,00:15:00,4183.81
5,2023-09-16,00:20:00,4166.43
6,2023-09-16,00:25:00,4135.74
7,2023-09-16,00:30:00,4113.43
8,2023-09-16,00:35:00,4086.84
9,2023-09-16,00:40:00,4064.81


In [16]:
plot(
    power_data, x=:Time, y=:Usage_MW, color=:Date,
    Layout(
        width=600,height=500,
        showlegend=false,
        xaxis=attr(dtick=24, tickformat="%H"),
        title="Daily Power Usage Trend"
    )
)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, legend, margin, showlegend, template, title, width, xaxis, and yaxis"

In [47]:
heating_data = CSV.read("Power Usage/House_Heating.csv", DataFrame);

In [53]:
heating_data

Row,Fuel Type,Value
,String,Int64
1,No Heating,60819
2,Heat Pump,723792
3,Electric Heater,675018
4,Fixed Gas Heater,179571
5,Portable Gas Heater,96570
6,Wood Burner,494841
7,Pellet Fire,15093
8,Coal Burner,18813
9,Other,45132


In [57]:
plot(
    heating_data, x=Symbol("Fuel Type"), y=:Value, color=Symbol("Fuel Type"), kind="bar",
    Layout(
        width=600,height=500,
        showlegend=false,
        title="Heating Usage Types (2018 Census)",
        xaxis_categoryorder="total descending"
    )
)

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, legend, margin, showlegend, template, title, width, xaxis, and yaxis"